* Config check script requirementsFor compatibility with inference branch config files* Are all of the primary sections to run the model present in the config?

* Do the primary sections have all of the necessary subsections?

* Are there stacked interventions (in a single scenario) that are overlapping in time? --> Warning

* Do the fields have the correct tabbing/spacing so that everything is valid?

* affected_geoids not effected_geoids field name

* Are there interventions that are unused? --> Warning

* Can we produce a figure that summarizes the intervention scenarios for easy review?

* Can we output a summary table of the outcomes values -- source and outcome name, values that were assigned

* Are all of the distributions valid? Do the distributions have valid parameters associated with them?

* Do intervention scenario start and end dates go outside of the simulation period? 

* is the config file a valid yml file? --> Error

* are there gaps in coverage (non-consecutive dates) in a stacked intervention -> Warning

* are the dates in the file actual dates (e.g., 2020-09-31 format) --> Error

* are there priors on unused parameters?

* are there hierarchical effects on unused parameters/interventions?

* is there inference on parameters that don't contribute to likelihood (e.g., inference on confirmation rate but deaths only likelihood)?

* Do interventions increase the value of the parameter that is being affected? --> warning

* Gap overlap

In [1]:
from SEIR import file_paths
from SEIR.utils import config
import yamllint


config.set_file('example.yml')

In [2]:
config

<Configuration: root>

In [3]:
yaml.safe_load('example.yml')

'example.yml'

In [15]:
import jsonschema
import datetime
import os
import yaml

# https://stackoverflow.com/questions/52210425/validate-datetime-value-using-python-jsonschema
def validate_with_datetime(schema, format_checker, resolver, instance):
  BaseVal = jsonschema.Draft7Validator

  # Build a new type checker
  def is_datetime(checker, inst):
    return isinstance(inst, datetime.date)
  date_check = BaseVal.TYPE_CHECKER.redefine('date', is_datetime)

  # Build a validator with the new type checker
  Validator = jsonschema.validators.extend(BaseVal, type_checker=date_check)

  # Run the new Validator
  Validator(schema=schema, format_checker=format_checker, resolver = resolver).validate(instance)

schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
     "type" : "object",
     "properties" : {
         "name" : {"type" : "string"},
         "start_date" : {"type" : "date"},
         "end_date" : {"type" : "date"},
         "nsimulations" : {"type":"number",
                          "exclusiveMinimum" : 0},
         "dt" : {"type":"number",
                "exclusiveMinimum" : 0},
         "spatial_setup":{"$ref": "file:spatial-setup.json"}
     },
     "required": [ "name", "start_date", "end_date", "nsimulations", "dt",
                 "spatial_setup"]
 }


In [34]:
with open('example.yml', 'r') as config_stream:
    #fdir = dirname(__file__)
    fdir = os.getcwd() + '/config-schemas/'
    validate_with_datetime(instance=yaml.safe_load(config_stream), 
                            schema=schema, 
                            resolver = jsonschema.RefResolver(base_uri='file://' + fdir, 
                                                              referrer=None),
                            format_checker=jsonschema.FormatChecker())

In [9]:
yaml_file = yaml.safe_load(open('example.yml', 'r'))
validate(instance=yaml_file, schema=schema)

NameError: name 'validate' is not defined

In [49]:
isinstance(datetime.date(2020,10,12), datetime.date)

True